## **Limpieza del dataset**

### 1. Importar las librerias que vamos a usar

In [89]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("../src"))
import cleaning as clean

Nos aseguramos que nuestro archivo .py de limpieza se ha importado correctamente, con todas las funciones.

In [90]:
clean.__file__

'c:\\Users\\adria\\OneDrive\\Escritorio\\DataScience\\Proyectos\\Modulo 1 - EDA\\src\\cleaning.py'

In [92]:
dir(clean)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__warningregistry__',
 'clean_client_satisfaction',
 'clean_columns',
 'clean_hourly_rate',
 'gender_clean',
 'imput_age',
 'is_active_clean',
 'pd']

### 2. Carga del dataset

In [93]:
df_original = pd.read_csv("../data/global_freelancers_raw.csv")
df = df_original.copy()

In [94]:
df.sample(5)

,freelancer_ID,name,gender,age,country,language,primary_skill,years_of_experience,hourly_rate (USD),rating,is_active,client_satisfaction
561,FL250562,Nicole Davis,Female,24.0,France,French,Cybersecurity,5.0,40,3.0,no,86%
879,FL250880,Mrs. Jessica Williams DVM,FEMALE,25.0,South Korea,Korean,AI,6.0,20,2.1,no,NaN
42,FL250043,Kaitlyn Schmidt,Female,35.0,Germany,German,DevOps,10.0,$50,0.0,no,NaN
284,FL250285,Ann Contreras,female,55.0,Japan,Japanese,Web Development,20.0,75,2.9,False,NaN
563,FL250564,Kathy Faulkner,FEMALE,20.0,Turkey,Turkish,Graphic Design,0.0,30,4.8,yes,68%


### 3.Limpieza del dataset.

Paso a paso de lo que tenemos que hacer:
1. Normalizar el nombre de las columnas: todos las letras en miniscula y eliminar la parte final de hourly_rate **(USD)**.
2. Limpieza y normalización de categorías:
    - gender: unificar variantes (male, M, MALE -> Male; female, F, FEMALE -> Female)
    - is_active: convertir todos los formatos a booleano (True/False)
3. Limpieza de valores mixtos en columnas numéricas:
    - hourly_rate: eliminar simbolos ($, USD), convertir a float y revisar outliers.
    - client_satisfaction: convertir "84%" -> 84.0, convertir a float
4. Tratamiento de valores *NaN*. Como ya tenemos el % de *NaN* por columna, debemos hacer lo siguiete:
    - Definir la estrategia a seguir por columna segun su relevancia:
        - Imputar (mediana o media) si la columna es critica y solo cuando el valor es objetivo, numérico y su significado no depende de interpretación ni condiciones externas.
        - Dejar los valores *NaN*, cuando son valores subjetivos o económicos, debido a que puede distorsional nuestro analisis al implementar valores ficticios.
5. Duplicados.
6. Conversión final de tipos.
7. Exportar el dataset con los datos limpios.
    
    

#### 3.1 Normalización de columnas.
Vamos a transformar todas las columnas a minuscula para hacer el trabajo de limpieza y la visualización más sencillo. Además de eliminar de la columna "hourly_rate (USD)" el final "(USD)".

In [95]:
df.columns

Index(['freelancer_ID', 'name', 'gender', 'age', 'country', 'language',
       'primary_skill', 'years_of_experience', 'hourly_rate (USD)', 'rating',
       'is_active', 'client_satisfaction'],
      dtype='object')

In [96]:
clean.clean_columns(df)
df.sample(3)

,freelancer_id,name,gender,age,country,language,primary_skill,years_of_experience,hourly_rate,rating,is_active,client_satisfaction
291,FL250292,Cathy Baldwin,Female,27.0,Japan,Japanese,Web Development,7.0,30,2.4,yes,66%
687,FL250688,Steven Moore,M,33.0,Japan,Japanese,Cybersecurity,NaN,USD 30,4.4,True,65%
362,FL250363,Arthur Vasquez,Male,25.0,Argentina,Spanish,Blockchain Development,5.0,$30,2.3,0,91%


### 3.2. Limpieza y normalización de categorías:
- gender: unificar variantes (male, M, MALE -> Male; female, F, FEMALE -> Female)

In [97]:
df["gender"].unique()

array(['f', 'FEMALE', 'male', 'F', 'female', 'm', 'MALE', 'Female', 'M',
       'Male'], dtype=object)

In [98]:
df["gender"]=df["gender"].apply(clean.gender_clean)
df["gender"].unique()

array(['Female', 'Male'], dtype=object)

- is_active: convertir todos los formatos a booleano (True/False)

In [99]:
df["is_active"].unique()

array(['0', '1', 'N', 'False', 'True', 'yes', 'Y', nan, 'no'],
      dtype=object)

In [100]:
df["is_active"].map(type).unique()

array([<class 'str'>, <class 'float'>], dtype=object)

In [101]:
df["is_active"] = df["is_active"].apply(clean.is_active_clean)
df["is_active"].unique()

array([False, True, nan], dtype=object)

In [102]:
df["is_active"].map(type).unique()

array([<class 'bool'>, <class 'float'>], dtype=object)

### 3. Limpieza de valores mixtos en columnas numéricas:
- hourly_rate: eliminar simbolos ($, USD), convertir a float y revisar outliers.

In [103]:
df["hourly_rate"].sample(5)

769     50
822     20
120     75
983    $75
758     40
Name: hourly_rate, dtype: object

In [104]:
df = clean.clean_hourly_rate(df)

In [105]:
df["hourly_rate"].value_counts()

hourly_rate
40.0     171
100.0    157
50.0     152
30.0     150
20.0     142
75.0     134
Name: count, dtype: int64

In [106]:
df["hourly_rate"].map(type).unique()

array([<class 'float'>], dtype=object)

- client_satisfaction: convertir "84%" -> 84.0, convertir a float.

In [107]:
df["client_satisfaction"].sample(3)

813    79%
887    76%
384    61%
Name: client_satisfaction, dtype: object

In [108]:
df = clean.clean_client_satisfaction (df)

In [109]:
df["client_satisfaction"].sample(3)

119    82.0
181    78.0
209    94.0
Name: client_satisfaction, dtype: float64

In [110]:
df["client_satisfaction"].map(type).unique()

array([<class 'float'>], dtype=object)

### 4. Tratamiento de valores *NaN*.
Tenemos las siguientes columnas con valores *NaN* y sus correspondientes *%*.
- "age" con 30 valores nulos de 1000 filas (3%)
- "years_of_experience" con 51 valores nulos de 1000 (5.1%)
- "hourly_rate" con 94 valores nulos de 1000 (9.4%)
- "rating" con 101 valores nulos de 1000 (10.1%)
- "is_active" con 89 valores nulos de 1000 (8.9%)
- "client_satisfaction" con 176 valores nulos de 1000 (17.6%)

Ahora que tenemos todas las columnas listadas con valores nulos, tenemos que clasificarlas segun su importancia:
1. Columnas principales. Estas son: "age", "years_of_experience" y "hourly_rate".
    - Imputación en la columna "age", al ser un valor objetivo y tener tan pocos valores nulos, podemos imputarlos usando su media o mediana. En este caso vamos a hacerlo con la mediana para que no distorsione mucho de la realidad y el analisis sea robusto (La imputación con mediana se recomienda porque la distribución puede tener outliers o valores extremos poco frecuentes).
    - La columna "years_of_experience", la dejaremos con los valores *NaN*, para evitar incoherencias como por ejemplo: "age: 22" - "years_of_experience: 9".
    - Dejaremos los valores NaN en la columna de "hourly_rate", debido a que al ser un valor económico, si imputamos podemos distosionar el analisis y añadir datos que varien las visualizaciones que haremos despues.
2. Columnas secundarias. Estas son "rating", "is_active" y "client_satisfaction".
    - Las columnas "rating" y "client_satisfaction" corresponden a un valor subjetivo que está sujeto a la opinión del cliente, en este caso, dejaremos los valores *NaN* para no sesgar el analisis con información "artificial".
    - La columna "is_active", al igual que las otras dos, dejaremos los valores *NaN* intactos, puesto que es información que el freelancer decide si compartir o no (la ausencia del valor puede ser en sí misma informativa sobre el comportamiento del freelancer), y forzar un True o False podría sesgar nuestro analisis. 

#### 4.1 Imputaciones
Vamos a imputar los valores nulos de la columnas "age".

Además una vez eliminados los nulos, convertiremos estas la columna de tipo *float64* a *int64*. Con esto conseguiremos:
- Ahorrar memoria.
- Evitar decimales innecasarios, debido a que todos los numeros son enteros.
- Hace que las visualizaciones y cálculos sean más naturales.

In [111]:
df = clean.imput_age(df)

c:\Users\adria\OneDrive\Escritorio\DataScience\Proyectos\Modulo 1 - EDA\src\cleaning.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["age"].fillna(mediana_age, inplace=True)


In [112]:
df["age"].isna().sum()

np.int64(0)

In [113]:
df["age"].describe()

count    1000.000000
mean       40.524000
std        11.762218
min        20.000000
25%        31.000000
50%        41.000000
75%        51.000000
max        60.000000
Name: age, dtype: float64

In [114]:
df["age"] = df["age"].astype("int64")

In [115]:
df.sample(3)

,freelancer_id,name,gender,age,country,language,primary_skill,years_of_experience,hourly_rate,rating,is_active,client_satisfaction
661,FL250662,Lynn French,Female,52,Canada,English,UI/UX Design,16.0,100.0,4.0,True,NaN
679,FL250680,Hannah Sanders,Female,29,South Korea,Korean,Graphic Design,0.0,20.0,NaN,True,99.0
463,FL250464,John Mathis,Male,26,Mexico,Spanish,Web Development,8.0,40.0,4.9,False,87.0


### 5. Duplicados.
En este caso los duplicados que tenemos vienen o bien de la columna "name", en la que se repiten nombres pero hemos identificado anteriormente que son de personas diferentes, o de las columnas categóricas que es normal que se repitan. Además no tenemos ninguna fila duplicada, por lo que no tenemos que hacer ninguna limpieza.

In [116]:
df.loc[df[["name","gender", "age", "country", "primary_skill"]].duplicated()]

,freelancer_id,name,gender,age,country,language,primary_skill,years_of_experience,hourly_rate,rating,is_active,client_satisfaction


### 6. Conversión final de tipos.
Durante el proceso de limpieza hemos convertido varias columnas:

- "age": float64 -> int64
- "years_of_experience": float64 -> int64
- "hourly_rate": string -> float64
- "is_active": string -> bool
- "client_satisfaction": string -> float64 

### 7. Exportar el dataset con los datos limpios.

In [117]:
df.to_csv("../data/freelancers_clean.csv", index=False)